## ETL PYTHON 

In [1]:
# Importar datos de Dataframe "Casos_positivos_de_COVID-19-Cund-Boy.csv" de Jupyternotebbok a MySQL

In [2]:
from pandas.io import sql
import pandas as pd
import sqlalchemy as db
from datetime import datetime, date
import numpy as np
from scipy import stats
import pymysql
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Leer el archivo CSV
df = pd.read_csv("Casos_positivos_de_COVID-19-Cund-Boy.csv", sep = ";")


In [4]:
df.head()

,id_case,id_municipality,name_municipality,id_department,name_department,age,gender,type_contagion,status,date_symptom,date_death,date_diagnosis,date_recovery
0,489982,25,CUNDINAMARCA,25899,ZIPAQUIRA,22,F,Comunitaria,Recuperado,2/08/2020,NaN,18/08/2020,20/12/2020
1,5932109,25,CUNDINAMARCA,25758,SOPO,50,M,Comunitaria,Recuperado,18/01/2022,NaN,2/02/2022,5/02/2022
2,3036330,25,CUNDINAMARCA,25099,BOJACA,67,M,Comunitaria,Recuperado,4/05/2021,NaN,8/05/2021,18/05/2021
3,1788659,25,CUNDINAMARCA,25151,CAQUEZA,51,F,Relacionado,Recuperado,4/01/2021,NaN,7/01/2021,18/01/2021
4,4715565,25,CUNDINAMARCA,25290,FUSAGASUGA,40,M,Comunitaria,Recuperado,8/07/2021,NaN,23/07/2021,25/07/2021


In [5]:
df.shape

(20000, 13)

In [6]:
## Exploración y transformación de datos de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id_case            20000 non-null  int64 
 1   id_municipality    20000 non-null  int64 
 2   name_municipality  20000 non-null  object
 3   id_department      20000 non-null  int64 
 4   name_department    20000 non-null  object
 5   age                20000 non-null  int64 
 6   gender             20000 non-null  object
 7   type_contagion     20000 non-null  object
 8   status             19857 non-null  object
 9   date_symptom       19873 non-null  object
 10  date_death         589 non-null    object
 11  date_diagnosis     19996 non-null  object
 12  date_recovery      19422 non-null  object
dtypes: int64(4), object(9)
memory usage: 2.0+ MB


In [7]:
#Quitar duplicados
df.drop_duplicates(subset='id_case',inplace=True)

In [8]:
# Verificación si habian duplicados
df.shape

(20000, 13)

In [9]:
# cambiar tipo de datos date

df['date_symptom'] = pd.to_datetime(df['date_symptom'])
df['date_death'] = pd.to_datetime(df['date_death'])
df['date_death'] = pd.to_datetime(df['date_diagnosis'])
df['date_recovery'] = pd.to_datetime(df['date_recovery'])

In [10]:
# Crear campo id_type_contagion 

condiciones = [
    (df['type_contagion'] == 'Comunitaria'),
    (df['type_contagion'] == 'Relacionado'),
    (df['type_contagion'] == 'Importado')
]

In [11]:
resultados = [1,2,3]

In [12]:
df['id_type_contagion'] = np.select(condiciones, resultados)

In [13]:
# Crear campo id_status

condicionessta = [
    (df['status'] == 'Activo'),
    (df['status'] == 'Fallecido'),
    (df['status'] == 'Recuperado'),
    (df['status'] == 'Sin_Informacion')    
]

In [14]:
resultadossta = [1,2,3,4]

In [15]:
df['id_status'] = np.select(condicionessta, resultadossta)

In [16]:
# Crear campo id_gender

condicionesgen = [
    (df['gender'] == 'F'),
    (df['gender'] == 'M')
]

In [17]:
resultadosgen = [1,2]

In [18]:
df['id_gender'] = np.select(condicionesgen, resultadosgen)

In [19]:
df

,id_case,id_municipality,name_municipality,id_department,name_department,age,gender,type_contagion,status,date_symptom,date_death,date_diagnosis,date_recovery,id_type_contagion,id_status,id_gender
0,489982,25,CUNDINAMARCA,25899,ZIPAQUIRA,22,F,Comunitaria,Recuperado,2020-02-08,2020-08-18,18/08/2020,2020-12-20,1,3,1
1,5932109,25,CUNDINAMARCA,25758,SOPO,50,M,Comunitaria,Recuperado,2022-01-18,2022-02-02,2/02/2022,2022-05-02,1,3,2
2,3036330,25,CUNDINAMARCA,25099,BOJACA,67,M,Comunitaria,Recuperado,2021-04-05,2021-08-05,8/05/2021,2021-05-18,1,3,2
3,1788659,25,CUNDINAMARCA,25151,CAQUEZA,51,F,Relacionado,Recuperado,2021-04-01,2021-07-01,7/01/2021,2021-01-18,2,3,1
4,4715565,25,CUNDINAMARCA,25290,FUSAGASUGA,40,M,Comunitaria,Recuperado,2021-08-07,2021-07-23,23/07/2021,2021-07-25,1,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,598195,15,BOYACA,15238,DUITAMA,46,F,Comunitaria,Recuperado,2020-12-08,2020-08-26,26/08/2020,2020-12-10,1,3,1
19996,4186035,15,BOYACA,15001,TUNJA,39,F,Relacionado,Recuperado,2021-12-06,2021-06-26,26/06/2021,2021-06-30,2,3,1
19997,4515219,15,BOYACA,15740,SIACHOQUE,24,M,Comunitaria,Recuperado,2021-06-26,2021-10-07,10/07/2021,2021-07-13,1,3,2
19998,5327425,15,BOYACA,15001,TUNJA,40,M,Relacionado,Recuperado,2021-12-24,2022-07-01,7/01/2022,2022-10-01,2,3,2


In [20]:
# Conectar con MYSQL y Cargar datos

In [21]:
database_username = 'root'
database_password = '1111'
database_ip       = 'localhost'
database_name     = 'prueba_practica'
database_connection = db.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.
                                             format(database_username,database_password,
                                                   database_ip, database_name))
connection = database_connection.connect()
metadata = db.MetaData()

In [22]:
df.to_sql('cases', connection)

20000